In [21]:
import os
import netket as nk
from netket.operator.spin import sigmax, sigmaz, sigmay
# from src.wilson import * 
from src.models import * 
import matplotlib.pyplot as plt
nk.config.netket_debug = True

## Parameters

In [15]:
L = 8

note hilbert space is on edges
there is no natural ordering to the hilbert space so we will have to impose one.
this is specified by wilson.ordered_edges.
as long as we respect this ordering across all implementations, things should work. Convention is vertical links first.

In [16]:
g = nk.graph.Square(L, pbc=True)
print(f'g.n_edges = {g.n_edges}, LxLx2 = {L**2 * 2}')
hi = nk.hilbert.Spin(s=1/2, N=g.n_edges)

g.n_edges = 128, LxLx2 = 128


The Z_2 gauge theory Hamiltonian is 
$$
H = -K \sum_{\square} \prod_{ij \in \square} \sigma_{ij}^z - g \prod_{ij} \sigma_{ij}^x
$$

In [19]:
K = 1
g_ham = 1
h = 0
edges = ordered_edges(g)

for i in g.nodes(): 
	l, r, u, d = plaquette_indices(i, L)
	h += -K * sigmaz(hi, l) * sigmaz(hi, d) * sigmaz(hi, r) * sigmaz(hi, u)

# for i in range(g.n_edges):
    # h += -g_ham * sigmax(hi, i)

# nk.exact.lanczos_ed(h, k=4)

In [20]:
genn = GENN(graph=g)
sampler = nk.sampler.MetropolisLocal(hi)
vstate = nk.vqs.MCState(sampler, model=genn, n_samples=1008)
optimizer = nk.optimizer.Sgd(learning_rate=0.05)
preconditioner = nk.optimizer.SR(diag_shift=0.1, holomorphic=False)
vmc = nk.driver.VMC(h, optimizer, variational_state=vstate, preconditioner=preconditioner)
log = nk.logging.RuntimeLog()
vmc.run(n_iter=10, out=log)

100%|██████████| 10/10 [00:05<00:00,  1.71it/s, Energy=-56.52+0.00j ± 0.12 [σ²=13.71, R̂=1.2358]]


(RuntimeLog():
  keys = ['Energy'],)

In [25]:
log['Energy']['Mean']

array([  3.62301587+0.j,   0.65873016+0.j,  -0.23809524+0.j,
        -1.53174603+0.j,  -3.5952381 +0.j,  -6.49603175+0.j,
       -14.28968254+0.j, -47.6031746 +0.j, -55.63095238+0.j,
       -56.51984127+0.j])